# Image Classification Model by CNN

In [1]:
# Importing libraries 

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

import pickle

import random
random.seed(0)

# Mounting Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Converting image to array:

def img2array(path, img_size= 32):

  img_array = cv2.imread(path)
  img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB) # Convert from BGR to RGB

  return cv2.resize(img_array, (img_size, img_size))


In [11]:
# Creating training data and testing data:

categories = ['NECKLACE', 'WRISTWATCH', 'EARRINGS', 'RINGS', 'BRACELET']

def createData(dir):

  data= []
  for cat in categories:    
    path = os.path.join(dir, cat)
    cat_num = categories.index(cat)

    for img in os.listdir(path):
      try:
        img_path = os.path.join(path, img)
        array = img2array(img_path, img_size= 32)  # Calling img2array function      
        data.append([array, cat_num])
      except Exception as e:
        pass

  return data


In [12]:
# Instatiating training and testing data

train_dir = '/content/drive/MyDrive/ImageClassification/training'
test_dir = '/content/drive/MyDrive/ImageClassification/test'

training_data = createData(train_dir)
testing_data = createData(test_dir)

In [13]:
print(f"Length of training data: {len(training_data)}\n\nLength of testing data: {len(testing_data)}")

Length of training data: 1564

Length of testing data: 250


In [16]:
# Pre-processing training data and testing data:

def pre_process(data, size= 32):  

  '''Shuffle the training data to reduce overfitting of model'''
  random.shuffle(data)

  '''Creating variables: feature set X and labels y'''
  X = []
  y = []

  for feat, lab in data:
    X.append(feat)
    y.append(lab)

  '''Converting feature set X into numpy array, so that it'll be able to pass to the neural network'''
  X = np.array(X).reshape(-1, size, size, 3) 
  y = np.array(y)

  '''Images are 8-bit value, which means it can have values between 0 and 255'''
  '''Normalizing feature set X by scaling between 0 and 1'''
  X = X/255.0

  return X, y
  


In [17]:
# Instatiating traing features and lables

X, y = pre_process(training_data)

# Instatiating testing features and lables

X_test, y_test = pre_process(testing_data)

In [18]:
# Compiling training model

def compile_model():

  '''Instantiating sequential class of keras'''
  model = Sequential()

  '''First convolution layer'''
  model.add(Conv2D(32, (3,3), input_shape = X.shape[1:]))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size= (2,2)))

  '''Second convolution layer'''
  model.add(Conv2D(64, (3,3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size= (2,2)))

  '''Dense layer'''
  model.add(Flatten())
  model.add(Dense(128))
  model.add(Activation('relu'))

  '''Output layer'''
  model.add(Dense(5))
  model.add(Activation('softmax'))

  '''Compiling model'''
  model.compile(loss= 'sparse_categorical_crossentropy', optimizer= 'adam', metrics = ['accuracy'])

  return model

model = compile_model() # Instantiating compiled model

In [19]:
# Summary

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 activation (Activation)     (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 activation_1 (Activation)   (None, 13, 13, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                    

In [20]:
# Fitting training model

def fit_model(epochs= 10, batch_size= 32):

  '''Creating Model Checkpoint'''
  file = 'final_model.h5'          
  checkpoint = ModelCheckpoint(file, monitor = 'val_accuracy', verbose= 1, save_best_only= True, mode= 'max')

  '''Fitting model''' 
  model.fit(X, y, epochs = epochs, batch_size= batch_size, validation_data = (X_test, y_test), callbacks = [checkpoint])

  return model, file

fit_model, final_model = fit_model()

Epoch 1/10
49/49 [==============================] - ETA: 0s - loss: 0.9859 - accuracy: 0.6049
Epoch 1: val_accuracy improved from -inf to 0.74000, saving model to final_model.h5
49/49 [==============================] - 11s 13ms/step - loss: 0.9859 - accuracy: 0.6049 - val_loss: 0.7272 - val_accuracy: 0.7400
Epoch 2/10
42/49 [========================>.....] - ETA: 0s - loss: 0.4207 - accuracy: 0.8534
Epoch 2: val_accuracy improved from 0.74000 to 0.82400, saving model to final_model.h5
49/49 [==============================] - 0s 5ms/step - loss: 0.4046 - accuracy: 0.8613 - val_loss: 0.5534 - val_accuracy: 0.8240
Epoch 3/10
42/49 [========================>.....] - ETA: 0s - loss: 0.3039 - accuracy: 0.8973
Epoch 3: val_accuracy improved from 0.82400 to 0.82800, saving model to final_model.h5
49/49 [==============================] - 0s 5ms/step - loss: 0.3003 - accuracy: 0.9003 - val_loss: 0.5811 - val_accuracy: 0.8280
Epoch 4/10
42/49 [========================>.....] - ETA: 0s - loss: 0.2

In [21]:
# Evaluating the model using testing dataset:

def evaluate_model():

  loss, accuracy = fit_model.evaluate(X_test, y_test, 32)

  print(f'\nTesting loss: {loss}\n\nTesting accuracy: {accuracy}')

evaluate_model()

8/8 [==============================] - 0s 4ms/step - loss: 0.4750 - accuracy: 0.8480

Testing loss: 0.4750211238861084

Testing accuracy: 0.8479999899864197


In [22]:
# Loading our final keras model from checkpoint 

def load():
  return load_model(final_model)

load_model = load()

In [23]:
# Processing new image

def process_new(img_file, size):

  array = img2array(img_file, img_size= size) # Calling img2array function
  new_array = np.array(array).reshape(size, size, 3)/255.0
  exp_array = np.expand_dims(new_array, axis=0) # Getting the expanded dimensions

  return exp_array

# Path of new input image

image = '/content/drive/MyDrive/ImageClassification/test_img.jpg'

new_array = process_new(image, size= 32) # Since, model is trained on image size = 32

In [24]:
# Creating function to predict classification of new image

def predict_class(input):

  predict = load_model.predict(input)
  
  class_index = np.argmax(predict)
  score = predict[0][class_index]

  print(f"\nThis image is predicted to be of {categories[class_index]} with confidence score = {score}")

In [25]:
# Predicting class of new image

predict_class(input = new_array)

1/1 [==============================] - 0s 161ms/step

This image is predicted to be of RINGS with confidence score = 0.5311072468757629


In [26]:
# Saving our final model in Drive as a pickel file

def save_as_pickle(path):
  with open(path, 'wb') as f:
    pickle.dump(load_model, f)

save_at_path = '/content/drive/MyDrive/ImageClassification/jewellery_class_cnn.pkl'
save_as_pickle(save_at_path)